## Importing libraries

In [ ]:
import numpy as np
import pandas as pd
import pyodbc
import unicodedata
import urllib
from sqlalchemy import create_engine
from sqlalchemy.types import VARCHAR, NVARCHAR, Integer, Float, DateTime, Boolean

### Establishing connection with SQL database

In [2]:
conn_str = (
    r"DRIVER={ODBC driver 17 for sql server};"
    r"SERVER=DESKTOP-47IJHFR\AFTERERROR;"
    r"DATABASE=Olist;"
    r"Trusted_connection=yes;"

)

In [3]:
cnxn = pyodbc.connect(conn_str)

In [4]:
tables = ["olist_geolocation_dataset",
          "olist_customers_dataset",
          "olist_sellers_dataset",
          "olist_products_dataset",
          "olist_orders_dataset",
          "olist_order_items_dataset",
          "olist_order_reviews_dataset",
          "olist_order_payments_dataset",
          "product_category_name_translation"
]

In [5]:
dfs = {}

for tb in tables:
    dfs[tb] = pd.read_sql(f"SELECT * FROM staging.{tb}", cnxn)

C:\Users\admin\AppData\Local\Temp\ipykernel_14704\4138702520.py:4: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  dfs[tb] = pd.read_sql(f"SELECT * FROM staging.{tb}", cnxn)


In [6]:
dfs.keys()

dict_keys(['olist_geolocation_dataset', 'olist_customers_dataset', 'olist_sellers_dataset', 'olist_products_dataset', 'olist_orders_dataset', 'olist_order_items_dataset', 'olist_order_reviews_dataset', 'olist_order_payments_dataset', 'product_category_name_translation'])

In [7]:
geolocation = dfs["olist_geolocation_dataset"]
customers = dfs["olist_customers_dataset"]
sellers = dfs["olist_sellers_dataset"]
products = dfs["olist_products_dataset"]
orders = dfs["olist_orders_dataset"]
order_items = dfs["olist_order_items_dataset"]
order_payments = dfs["olist_order_payments_dataset"]
order_reviews = dfs["olist_order_reviews_dataset"]
cat_translation = dfs["product_category_name_translation"]

In [8]:
del dfs

In [9]:
for name, df in [("geolocation", geolocation), 
                 ("customers", customers), 
                 ("sellers", sellers), 
                 ("products", products),
                 ("orders", orders),
                 ("order_items", order_items),
                 ("order_payments", order_payments),
                 ("order_reviews", order_reviews),
                 ("cat_translation", cat_translation)]:
    
    print(name, df.shape)
    print(df.isnull().sum(), "\n")


geolocation (1000163, 5)
geolocation_zip_code_prefix    0
geolocation_lat                0
geolocation_lng                0
geolocation_city               0
geolocation_state              0
dtype: int64 

customers (99441, 5)
customer_id                 0
customer_unique_id          0
customer_zip_code_prefix    0
customer_city               0
customer_state              0
dtype: int64 

sellers (3095, 4)
seller_id                 0
seller_zip_code_prefix    0
seller_city               0
seller_state              0
dtype: int64 

products (32951, 9)
product_id                      0
product_category_name         610
product_name_lenght           610
product_description_lenght    610
product_photos_qty            610
product_weight_g                2
product_length_cm               2
product_height_cm               2
product_width_cm                2
dtype: int64 

orders (99441, 8)
order_id                            0
customer_id                         0
order_status                 

**We have null values in products, orders and order_reviews table**

In [10]:
# Table cat_translation needs its headers fixed

new_header = cat_translation.iloc[0]            # Assigning first row as the header names
cat_translation = cat_translation[1:]           # Eliminating first row from the table
cat_translation.columns = new_header.tolist()   # ASsigning header to the table
cat_translation.reset_index(drop=True, inplace=True)    # Dropping the index column and resetting the index
cat_translation.head()

,product_category_name,product_category_name_english
0,beleza_saude,health_beauty
1,informatica_acessorios,computers_accessories
2,automotivo,auto
3,cama_mesa_banho,bed_bath_table
4,moveis_decoracao,furniture_decor


### Analysing Geolocation table

In [11]:
# Primary analysis
for column in geolocation:
    print(f"{column} : {len(geolocation[column].unique())}")

geolocation_zip_code_prefix : 19015
geolocation_lat : 717372
geolocation_lng : 717615
geolocation_city : 8011
geolocation_state : 27


In [12]:
geolocation.head() # General view

,geolocation_zip_code_prefix,geolocation_lat,geolocation_lng,geolocation_city,geolocation_state
0,01037,-23.54562128115268,-46.63929204800168,sao paulo,SP
1,01046,-23.546081127035535,-46.64482029837157,sao paulo,SP
2,01046,-23.54612896641469,-46.64295148361138,sao paulo,SP
3,01041,-23.5443921648681,-46.63949930627844,sao paulo,SP
4,01035,-23.541577961711493,-46.64160722329613,sao paulo,SP


In [13]:
geolocation["geolocation_city"].unique()    # Unique values of city

array(['sao paulo', 'são paulo', 'sao bernardo do campo', ..., 'ciríaco',
       'estação', 'vila lângaro'], dtype=object)

In [14]:
# Defining a function to normalize city names
def normalize_city(name):
    if pd.isna(name):
        return name

    nfkd = unicodedata.normalize('NFKD', name)      # Decompose unicode characters
    no_accents = ''.join(c for c in nfkd if not unicodedata.combining(c))
    return no_accents.strip().lower().title()
    

In [15]:
# Creating a normalized column of city names
geolocation["city_clean"] = geolocation["geolocation_city"].apply(normalize_city)

In [16]:
geolocation.head()  # General view

,geolocation_zip_code_prefix,geolocation_lat,geolocation_lng,geolocation_city,geolocation_state,city_clean
0,01037,-23.54562128115268,-46.63929204800168,sao paulo,SP,Sao Paulo
1,01046,-23.546081127035535,-46.64482029837157,sao paulo,SP,Sao Paulo
2,01046,-23.54612896641469,-46.64295148361138,sao paulo,SP,Sao Paulo
3,01041,-23.5443921648681,-46.63949930627844,sao paulo,SP,Sao Paulo
4,01035,-23.541577961711493,-46.64160722329613,sao paulo,SP,Sao Paulo


In [17]:
geolocation.info()  # General observation

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000163 entries, 0 to 1000162
Data columns (total 6 columns):
 #   Column                       Non-Null Count    Dtype 
---  ------                       --------------    ----- 
 0   geolocation_zip_code_prefix  1000163 non-null  object
 1   geolocation_lat              1000163 non-null  object
 2   geolocation_lng              1000163 non-null  object
 3   geolocation_city             1000163 non-null  object
 4   geolocation_state            1000163 non-null  object
 5   city_clean                   1000163 non-null  object
dtypes: object(6)
memory usage: 45.8+ MB


In [18]:
geolocation["geolocation_city"] = geolocation["city_clean"]     # Normalizing the geolocation_city column

In [19]:
del geolocation["city_clean"]   # Deleting city_clean column as it is redundant

In [20]:
geolocation["geolocation_city"].nunique()   # Checking unique count of geolocation_city

5967

In [21]:
# Changing the data types
geolocation["geolocation_zip_code_prefix"] = geolocation["geolocation_zip_code_prefix"].astype(int)
geolocation["geolocation_lat"] = pd.to_numeric(geolocation["geolocation_lat"], errors="coerce")
geolocation["geolocation_lng"] = pd.to_numeric(geolocation["geolocation_lng"], errors="coerce")

In [22]:
# Creating unique zip_code, city and state records
geo_grp = geolocation.groupby(
    ["geolocation_zip_code_prefix", "geolocation_city", "geolocation_state"]
).size().reset_index(name="count")

geo_grp.head()

,geolocation_zip_code_prefix,geolocation_city,geolocation_state,count
0,1001,Sao Paulo,SP,26
1,1002,Sao Paulo,SP,13
2,1003,Sao Paulo,SP,17
3,1004,Sao Paulo,SP,22
4,1005,Sao Paulo,SP,25


In [23]:
del geo_grp["count"]    # Deleting the count column

In [24]:
geo_grp.shape   # Shape of grouped dataframe

(19616, 3)

### Analysing Customers table

In [25]:
# Primary analysis
for column in customers:
    print(f"{column} : {len(customers[column].unique())}")

customer_id : 99441
customer_unique_id : 96096
customer_zip_code_prefix : 14994
customer_city : 4119
customer_state : 27


In [26]:
# Unique values in custoemr_id column is the same as the lenght of the Custoemrs table. Hence, unique value for each row

In [27]:
# Normalize customer_id column
customers["customer_id"] = (
    customers["customer_id"]
    .str.lower()
    .str.strip()
    .str.replace(r"\s+", "", regex = True)
)

# Normalize customer_unique_id column
customers["customer_unique_id"] =( 
    customers["customer_unique_id"]
    .str.lower()
    .str.strip()
    .str.replace(r"\s+", "", regex=True)
)

# Convert to integer, catching any bad values
customers["customer_zip_code_prefix"] = (
    pd.to_numeric(customers["customer_zip_code_prefix"],
                  errors = "coerce")
)

# Normalize customer_city column
customers["customer_city"] = (
    customers["customer_city"]
    .str.lower()
    .str.strip()
    .str.replace(r"\s+", " ", regex=True)
    .str.title()
)

# Normalize custoemr_state column
customers["customer_state"] = (
    customers["customer_state"]
    .str.upper()
    .str.strip()
)

In [28]:
customers["customer_zip_code_prefix"].isna().sum() # Checking Null values in zip_code_prefix

0

In [29]:
# No null values detected. Hence, the column is clean

In [30]:
customers.head()    # General view

,customer_id,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state
0,06b8999e2fba1a1fbc88172c00ba8bc7,861eff4711a542e4b93843c6dd7febb0,14409,Franca,SP
1,18955e83d337fd6b2def6b18a428ac77,290c77bc529b7ac935b93aa66c333dc3,9790,Sao Bernardo Do Campo,SP
2,4e7b3e00288586ebd08712fdd0374a03,060e732b5b29e8181a18229c7b0b2b5e,1151,Sao Paulo,SP
3,b2b6027bc5c5109e529d4dc6358b12c3,259dac757896d24d7702b9acbbff3f3c,8775,Mogi Das Cruzes,SP
4,4f2d8ab171c80ec8364f7c12e35b23ad,345ecd01c38d18a9036ed96c73b8d066,13056,Campinas,SP


In [31]:
customers.info()    # General observation

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99441 entries, 0 to 99440
Data columns (total 5 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   customer_id               99441 non-null  object
 1   customer_unique_id        99441 non-null  object
 2   customer_zip_code_prefix  99441 non-null  int64 
 3   customer_city             99441 non-null  object
 4   customer_state            99441 non-null  object
dtypes: int64(1), object(4)
memory usage: 3.8+ MB


In [32]:
customers["clean_city"] = customers["customer_city"].apply(normalize_city)

In [33]:
customers["clean_city"].nunique()

4119

In [34]:
# Unique count remains the same after normalization
# So clean_city becomes redundant and can be deleted

In [35]:
del customers["clean_city"]

### Analyzing Sellers table

In [36]:
# Primary analysis
for column in sellers:
    print(f"{column} : {len(sellers[column].unique())}")

print("Shape: ", sellers.shape)

seller_id : 3095
seller_zip_code_prefix : 2246
seller_city : 611
seller_state : 23
Shape:  (3095, 4)


In [37]:
# Unique count of seller_id is the same as the length of seller table. Hence, unique value for each row

In [38]:
sellers.head()

,seller_id,seller_zip_code_prefix,seller_city,seller_state
0,3442f8959a84dea7ee197c632cb2df15,13023,campinas,SP
1,d1b65fc7debc3361ea86b5f14c68d2e2,13844,mogi guacu,SP
2,ce3ad9de960102d0677a81f5d0bb7b2d,20031,rio de janeiro,RJ
3,c0f3eea2e14555b6faeea3dd58c1b1c3,04195,sao paulo,SP
4,51a04a8a6bdcb23deccc82b0b80742cf,12914,braganca paulista,SP


In [39]:
# Normalize seller_id column
sellers["seller_id"] = (
    sellers["seller_id"]
    .str.lower()
    .str.strip()
    .str.replace(r"\s+", "", regex = True)
)

# Convert seller_zip_code_prefix to int, coercing errors

sellers["seller_zip_code_prefix"] = (
    pd.to_numeric(sellers["seller_zip_code_prefix"],
                  errors= "coerce")
)

# Normalize sller_city column
sellers["seller_city"] = (
    sellers["seller_city"]
    .str.lower()
    .str.strip()
    .str.replace(r"\s+", " ", regex=True)
    .str.title()
)

#Normalize seller_state column
sellers["seller_state"] = (
    sellers["seller_state"]
    .str.upper()
    .str.strip()
)

In [40]:
sellers["seller_zip_code_prefix"].isna().sum() # Check for null values in zip_cod_prefix column

0

In [41]:
sellers.info()  # General observation

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3095 entries, 0 to 3094
Data columns (total 4 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   seller_id               3095 non-null   object
 1   seller_zip_code_prefix  3095 non-null   int64 
 2   seller_city             3095 non-null   object
 3   seller_state            3095 non-null   object
dtypes: int64(1), object(3)
memory usage: 96.8+ KB


In [42]:
sellers["clean_city"] = sellers["seller_city"].apply(normalize_city)    # Creating a normalized column of city

In [43]:
sellers["clean_city"].nunique() # Checking unique counts of normalized city column

606

In [44]:
# Earlier there were 611 unique counts for seller_city and there are 606 unique counts for clean_city
# That means seller_city has duplicate values but not visibly duplicate because of non-english characters
# So will replace seller_city with clean_city


In [45]:
sellers["seller_city"] = sellers["clean_city"]  # Swappign seller_city with clean_city values

In [46]:
del sellers["clean_city"]   # Delete clean_city column as it is redundant

In [47]:
sellers["seller_city"].nunique()    # Sanity check for unique count

606

### Analyzing Products Table

In [48]:
products.head()     # General view

,product_id,product_category_name,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm
0,1e9e8ef04dbcff4541ed26657ea517e5,perfumaria,40,287,1,225,16,10,14
1,3aa071139cb16b67ca9e5dea641aaa2f,artes,44,276,1,1000,30,18,20
2,96bd76ec8810374ed1b65e291975717f,esporte_lazer,46,250,1,154,18,9,15
3,cef67bcfe19066a932b7673e239eb23d,bebes,27,261,1,371,26,4,26
4,9dc1a7de274444849c219cff195d0b71,utilidades_domesticas,37,402,4,625,20,17,13


In [49]:
# Primary Analysis
for column in products:
    print(f"{column} : {len(products[column].unique())}")
print("Shape:", products.shape)

product_id : 32951
product_category_name : 74
product_name_lenght : 67
product_description_lenght : 2961
product_photos_qty : 20
product_weight_g : 2205
product_length_cm : 100
product_height_cm : 103
product_width_cm : 96
Shape: (32951, 9)


In [50]:
# Unique count of product_id is same as the length of products table

# Need to change the name of product_despcritpion_lenght and product_name_lenght column

In [51]:
# Renaming the columns
products.rename(
    columns={
        "product_name_lenght" : "product_name_length",
        "product_description_lenght": "product_description_length"
        }, inplace=True
    )

In [52]:
products.head()

,product_id,product_category_name,product_name_length,product_description_length,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm
0,1e9e8ef04dbcff4541ed26657ea517e5,perfumaria,40,287,1,225,16,10,14
1,3aa071139cb16b67ca9e5dea641aaa2f,artes,44,276,1,1000,30,18,20
2,96bd76ec8810374ed1b65e291975717f,esporte_lazer,46,250,1,154,18,9,15
3,cef67bcfe19066a932b7673e239eb23d,bebes,27,261,1,371,26,4,26
4,9dc1a7de274444849c219cff195d0b71,utilidades_domesticas,37,402,4,625,20,17,13


In [53]:
# Merging the English category names with the product table

products = products.merge(
    cat_translation,
    on = "product_category_name",
    how = "left"
)

In [54]:
# As observed earlier there are some null values in the prouducts table

In [55]:
products.isnull().sum() # Observing the null values

product_id                         0
product_category_name            610
product_name_length              610
product_description_length       610
product_photos_qty               610
product_weight_g                   2
product_length_cm                  2
product_height_cm                  2
product_width_cm                   2
product_category_name_english    623
dtype: int64

In [56]:
# Analysing null values in the products table
products[
    (products["product_category_name"].isna()) | 
    (products["product_name_length"].isna()) | 
    (products["product_description_length"].isna()) |
    (products["product_category_name_english"].isna()) |
    (products["product_length_cm"].isna()) |
    (products["product_width_cm"].isna()) |
    (products["product_height_cm"].isna())
]

,product_id,product_category_name,product_name_length,product_description_length,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm,product_category_name_english
105,a41e356c76fab66334f36de622ecbd3a,None,None,None,None,650,17,14,12,NaN
128,d8dee61c2034d6d075997acef1870e9b,None,None,None,None,300,16,7,20,NaN
145,56139431d72cd51f19eb9f7dae4d1617,None,None,None,None,200,20,20,20,NaN
154,46b48281eb6d663ced748f324108c733,None,None,None,None,18500,41,30,41,NaN
197,5fb61f482620cb672f5e586bb132eae9,None,None,None,None,300,35,7,12,NaN
...,...,...,...,...,...,...,...,...,...,...
32515,b0a0c5dd78e644373b199380612c350a,None,None,None,None,1800,30,20,70,NaN
32589,10dbe0fbaa2c505123c17fdc34a63c56,None,None,None,None,800,30,10,23,NaN
32616,bd2ada37b58ae94cc838b9c0569fecd8,None,None,None,None,200,21,8,16,NaN
32772,fa51e914046aab32764c41356b9d4ea4,None,None,None,None,1300,45,16,45,NaN


In [57]:
# Delving into product_category_name_english null values
products[(products["product_category_name_english"].isna()) & (~products["product_category_name"].isna())]

,product_id,product_category_name,product_name_length,product_description_length,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm,product_category_name_english
1628,0105b5323d24fc655f73052694dbbb3a,pc_gamer,59,621,4,2839,19,16,18,NaN
5821,6fd83eb3e0799b775e4f946bd66657c0,portateis_cozinha_e_preparadores_de_alimentos,52,280,1,1200,25,33,25,NaN
7325,5d923ead886c44b86845f69e50520c3e,portateis_cozinha_e_preparadores_de_alimentos,58,284,1,1200,25,33,25,NaN
7478,6727051471a0fc4a0e7737b57bff2549,pc_gamer,60,1532,3,650,16,22,20,NaN
8819,bed164d9d628cf0593003389c535c6e0,portateis_cozinha_e_preparadores_de_alimentos,54,382,2,850,30,21,22,NaN
11039,1220978a08a6b29a202bc015b18250e9,portateis_cozinha_e_preparadores_de_alimentos,46,280,1,1200,25,33,25,NaN
14266,ae62bb0f95af63d64eae5f93dddea8d3,portateis_cozinha_e_preparadores_de_alimentos,59,927,1,10600,40,20,38,NaN
16182,1954739d84629e7323a4295812a3e0ec,portateis_cozinha_e_preparadores_de_alimentos,58,792,4,750,30,30,30,NaN
16930,dbe520fb381ad695a7e1f2807d20c765,pc_gamer,60,840,6,800,18,22,22,NaN
17800,c7a3f1a7f9eef146cc499368b578b884,portateis_cozinha_e_preparadores_de_alimentos,52,1372,5,7350,40,30,23,NaN


In [58]:
# So there are 13 instances where product_category_name is not null while it's English counterpart is null
# There are 74 unique counts of product_category_name in products table, while there are only 72 unique counts for product_category_name_English in cat_translation table
# So everything adds up

In [59]:
# Imputing null values in product_category_name_english column

products.loc[
    products["product_category_name"]=="pc_gamer",
    "product_category_name_english"
    ] = "pc_gamer"

products.loc[
    products["product_category_name"] == "portateis_cozinha_e_preparadores_de_alimentos",
    "product_category_name_english"
] = "Portable Kitchen Food Prepares"

In [60]:
products["product_category_name_english"].isna().sum()  # Checking null values

610

In [61]:
# The null values have reduced from 623 to 610 after imputation

In [62]:
products.columns    # Columns in products tables

Index(['product_id', 'product_category_name', 'product_name_length',
       'product_description_length', 'product_photos_qty', 'product_weight_g',
       'product_length_cm', 'product_height_cm', 'product_width_cm',
       'product_category_name_english'],
      dtype='object')

In [63]:
# Converting columns to numerical data type
num_col = ['product_description_length', 'product_photos_qty', 'product_weight_g',
       'product_length_cm', 'product_height_cm', 'product_width_cm',]

for col in num_col:
    products[col] = pd.to_numeric(products[col])

In [64]:
# Separating columns from num_col into another list
num_col_weight_dim = ['product_weight_g',
       'product_length_cm', 'product_height_cm', 'product_width_cm']

In [65]:
# Imputing the null values with median
medians = products[num_col_weight_dim].median()
products[num_col_weight_dim] = products[num_col_weight_dim].fillna(medians)

In [66]:
# Imputing categorical columns with null values and numerical columns with 0
products.fillna({
    "product_category_name": "unknown",
    "product_name_length" : 0,
    "product_description_length" : 0,
    "product_photos_qyt": 0,
    "product_category_name_english": "unknown"
    }, inplace=True
)

In [67]:
products.isnull().sum() # checking null values after imputation

product_id                         0
product_category_name              0
product_name_length                0
product_description_length         0
product_photos_qty               610
product_weight_g                   0
product_length_cm                  0
product_height_cm                  0
product_width_cm                   0
product_category_name_english      0
dtype: int64

In [68]:
# Spelling mistake has resulted in null values still being present in product_photos_qty
products["product_photos_qty"] = products["product_photos_qty"].fillna(0)

In [69]:
products.isnull().sum()     #Checking null values

product_id                       0
product_category_name            0
product_name_length              0
product_description_length       0
product_photos_qty               0
product_weight_g                 0
product_length_cm                0
product_height_cm                0
product_width_cm                 0
product_category_name_english    0
dtype: int64

In [70]:
# The products table is clean with no null values

In [71]:
products.head()     # General view

,product_id,product_category_name,product_name_length,product_description_length,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm,product_category_name_english
0,1e9e8ef04dbcff4541ed26657ea517e5,perfumaria,40,287.0,1.0,225.0,16.0,10.0,14.0,perfumery
1,3aa071139cb16b67ca9e5dea641aaa2f,artes,44,276.0,1.0,1000.0,30.0,18.0,20.0,art
2,96bd76ec8810374ed1b65e291975717f,esporte_lazer,46,250.0,1.0,154.0,18.0,9.0,15.0,sports_leisure
3,cef67bcfe19066a932b7673e239eb23d,bebes,27,261.0,1.0,371.0,26.0,4.0,26.0,baby
4,9dc1a7de274444849c219cff195d0b71,utilidades_domesticas,37,402.0,4.0,625.0,20.0,17.0,13.0,housewares


In [72]:
# Normalize product_category_name column
products["product_category_name"] = (
    products["product_category_name"]
    .str.lower()
    .str.strip()
    .str.replace(r"\s+", " ", regex=True)
)

# Normalize product_category_name_english column
products["product_category_name_english"] = (
    products["product_category_name_english"]
    .str.lower()
    .str.strip()
    .str.replace(r"\s+", " ", regex=True)
)

In [73]:
products.info()     # General observation

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32951 entries, 0 to 32950
Data columns (total 10 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   product_id                     32951 non-null  object 
 1   product_category_name          32951 non-null  object 
 2   product_name_length            32951 non-null  object 
 3   product_description_length     32951 non-null  float64
 4   product_photos_qty             32951 non-null  float64
 5   product_weight_g               32951 non-null  float64
 6   product_length_cm              32951 non-null  float64
 7   product_height_cm              32951 non-null  float64
 8   product_width_cm               32951 non-null  float64
 9   product_category_name_english  32951 non-null  object 
dtypes: float64(6), object(4)
memory usage: 2.5+ MB


### Analyzing Orders table

In [74]:
orders.head()       # General view

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18 00:00:00
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13 00:00:00
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49,2018-08-08 08:55:23,2018-08-08 13:50:00,2018-08-17 18:06:29,2018-09-04 00:00:00
3,949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,delivered,2017-11-18 19:28:06,2017-11-18 19:45:59,2017-11-22 13:39:59,2017-12-02 00:28:42,2017-12-15 00:00:00
4,ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,delivered,2018-02-13 21:18:39,2018-02-13 22:20:29,2018-02-14 19:46:34,2018-02-16 18:17:02,2018-02-26 00:00:00


In [75]:
# Primary analysis
for column in orders:
    print(f"{column} : {len(orders[column].unique())}")
print("Shape:", orders.shape)

order_id : 99441
customer_id : 99441
order_status : 8
order_purchase_timestamp : 98875
order_approved_at : 90734
order_delivered_carrier_date : 81019
order_delivered_customer_date : 95665
order_estimated_delivery_date : 459
Shape: (99441, 8)


In [76]:
# order_id and customer_id are unique for each row

In [77]:
orders.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99441 entries, 0 to 99440
Data columns (total 8 columns):
 #   Column                         Non-Null Count  Dtype 
---  ------                         --------------  ----- 
 0   order_id                       99441 non-null  object
 1   customer_id                    99441 non-null  object
 2   order_status                   99441 non-null  object
 3   order_purchase_timestamp       99441 non-null  object
 4   order_approved_at              99281 non-null  object
 5   order_delivered_carrier_date   97658 non-null  object
 6   order_delivered_customer_date  96476 non-null  object
 7   order_estimated_delivery_date  99441 non-null  object
dtypes: object(8)
memory usage: 6.1+ MB


In [78]:
# order_approved_at, order_delivered_carrier_date and order_deliver_customer_date have null values

In [79]:
orders.columns      # columns in roders table

Index(['order_id', 'customer_id', 'order_status', 'order_purchase_timestamp',
       'order_approved_at', 'order_delivered_carrier_date',
       'order_delivered_customer_date', 'order_estimated_delivery_date'],
      dtype='object')

In [80]:
# Converting date columns into datetime format
date_col = ['order_purchase_timestamp',
       'order_approved_at', 'order_delivered_carrier_date',
       'order_delivered_customer_date', 'order_estimated_delivery_date']

for col in date_col:
    orders[col] = pd.to_datetime(
        orders[col],
        errors = "coerce",
        infer_datetime_format = "True"
)

C:\Users\admin\AppData\Local\Temp\ipykernel_14704\422517132.py:7: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  orders[col] = pd.to_datetime(
C:\Users\admin\AppData\Local\Temp\ipykernel_14704\422517132.py:7: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  orders[col] = pd.to_datetime(
C:\Users\admin\AppData\Local\Temp\ipykernel_14704\422517132.py:7: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-

In [81]:
orders.dtypes       # Checking data types

order_id                                 object
customer_id                              object
order_status                             object
order_purchase_timestamp         datetime64[ns]
order_approved_at                datetime64[ns]
order_delivered_carrier_date     datetime64[ns]
order_delivered_customer_date    datetime64[ns]
order_estimated_delivery_date    datetime64[ns]
dtype: object

In [82]:
orders.head()       # General view

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49,2018-08-08 08:55:23,2018-08-08 13:50:00,2018-08-17 18:06:29,2018-09-04
3,949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,delivered,2017-11-18 19:28:06,2017-11-18 19:45:59,2017-11-22 13:39:59,2017-12-02 00:28:42,2017-12-15
4,ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,delivered,2018-02-13 21:18:39,2018-02-13 22:20:29,2018-02-14 19:46:34,2018-02-16 18:17:02,2018-02-26


In [83]:
orders["order_status"].unique()     # Checking unique values

array(['delivered', 'invoiced', 'shipped', 'processing', 'unavailable',
       'canceled', 'created', 'approved'], dtype=object)

In [84]:
# Normalizing order_id column
orders["order_id"] = (
    orders["order_id"]
    .str.lower()
    .str.strip()
    .str.replace(r"\s+", "", regex=True)
)

# Normalizing customer_id column
orders["customer_id"] = (
    orders["customer_id"]
    .str.lower()
    .str.strip()
    .str.replace(r"\s+", "", regex=True)
)

# Normalizing order_status column
orders["order_status"] = (
    orders["order_status"]
    .str.lower()
    .str.strip()
    .str.replace(r"\s+", "", regex=True)
)

In [85]:
orders.isnull().sum()       # Checking null values

order_id                            0
customer_id                         0
order_status                        0
order_purchase_timestamp            0
order_approved_at                 160
order_delivered_carrier_date     1783
order_delivered_customer_date    2965
order_estimated_delivery_date       0
dtype: int64

In [86]:
# Analyzing null values
orders[
    (orders["order_approved_at"].isna()) |
    (orders["order_delivered_carrier_date"].isna()) |
    (orders["order_delivered_customer_date"].isna())
]

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date
6,136cce7faa42fdb2cefd53fdc79a6098,ed0271e0b7da060a393796590e7b737a,invoiced,2017-04-11 12:22:08,2017-04-13 13:25:17,NaT,NaT,2017-05-09
44,ee64d42b8cf066f35eac1cf57de1aa85,caded193e8e47b8362864762a83db3c5,shipped,2018-06-04 16:44:48,2018-06-05 04:31:18,2018-06-05 14:32:00,NaT,2018-06-28
103,0760a852e4e9d89eb77bf631eaaf1c84,d2a79636084590b7465af8ab374a8cf5,invoiced,2018-08-03 17:44:42,2018-08-07 06:15:14,NaT,NaT,2018-08-21
128,15bed8e2fec7fdbadb186b57c46c92f2,f3f0e613e0bdb9c7cee75504f0f90679,processing,2017-09-03 14:22:03,2017-09-03 14:30:09,NaT,NaT,2017-10-03
154,6942b8da583c2f9957e990d028607019,52006a9383bf149a4fb24226b173106f,shipped,2018-01-10 11:33:07,2018-01-11 02:32:30,2018-01-11 19:39:23,NaT,2018-02-07
...,...,...,...,...,...,...,...,...
99283,3a3cddda5a7c27851bd96c3313412840,0b0d6095c5555fe083844281f6b093bb,canceled,2018-08-31 16:13:44,NaT,NaT,NaT,2018-10-01
99313,e9e64a17afa9653aacf2616d94c005b8,b4cd0522e632e481f8eaf766a2646e86,processing,2018-01-05 23:07:24,2018-01-09 07:18:05,NaT,NaT,2018-02-06
99347,a89abace0dcc01eeb267a9660b5ac126,2f0524a7b1b3845a1a57fcf3910c4333,canceled,2018-09-06 18:45:47,NaT,NaT,NaT,2018-09-27
99348,a69ba794cc7deb415c3e15a0a3877e69,726f0894b5becdf952ea537d5266e543,unavailable,2017-08-23 16:28:04,2017-08-28 15:44:47,NaT,NaT,2017-09-15


In [87]:
# Calculating time difference between order_purchase_timestamp and order_approved_at
mask = orders["order_approved_at"].notna()
time_diff = (
    orders.loc[mask, "order_approved_at"]
    - orders.loc[mask, "order_purchase_timestamp"]
) 

In [88]:
# Calculating the median time difference between order_purchase_time and order_approved_at
median_time_diff = time_diff.median()
median_time_diff

Timedelta('0 days 00:20:36')

In [89]:
# Imputing null values for order_approved_at
orders["order_approved_at"] = orders["order_approved_at"].fillna(orders["order_purchase_timestamp"] + median_time_diff)

In [90]:
orders["order_approved_at"].isnull().sum()  # Sanity check for null values

0

In [91]:
# No null values remain in order_approved_at column

In [92]:
# Similarly, we will impute the null values in order_delivered_carrier_date and order_delivered_customer_date

In [93]:
# Imputing null values for order_delivered_carrier_date column
mask_carrier = orders["order_delivered_carrier_date"].notna()
time_diff_carrier = (
    orders.loc[mask_carrier, "order_delivered_carrier_date"] -
    orders.loc[mask_carrier, "order_purchase_timestamp"]
)

median_time_diff_carrier = time_diff_carrier.median()

orders["order_delivered_carrier_date"] = orders["order_delivered_carrier_date"].fillna(orders["order_purchase_timestamp"] + median_time_diff_carrier)

# Imputing null values for order_delivered_customer_date column
mask_customer = orders["order_delivered_customer_date"].notna()
time_diff_customer = (
    orders.loc[mask_customer, "order_delivered_customer_date"] -
    orders.loc[mask_customer, "order_purchase_timestamp"]
)

median_time_diff_customer = time_diff_customer.median()

orders["order_delivered_customer_date"] = orders["order_delivered_customer_date"].fillna(orders["order_purchase_timestamp"] + median_time_diff_customer)

In [94]:
orders.isnull().sum()       # Sanity check for null values

order_id                         0
customer_id                      0
order_status                     0
order_purchase_timestamp         0
order_approved_at                0
order_delivered_carrier_date     0
order_delivered_customer_date    0
order_estimated_delivery_date    0
dtype: int64

In [95]:
# No null values present in Orders table

In [96]:
orders.info()       # General observation

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99441 entries, 0 to 99440
Data columns (total 8 columns):
 #   Column                         Non-Null Count  Dtype         
---  ------                         --------------  -----         
 0   order_id                       99441 non-null  object        
 1   customer_id                    99441 non-null  object        
 2   order_status                   99441 non-null  object        
 3   order_purchase_timestamp       99441 non-null  datetime64[ns]
 4   order_approved_at              99441 non-null  datetime64[ns]
 5   order_delivered_carrier_date   99441 non-null  datetime64[ns]
 6   order_delivered_customer_date  99441 non-null  datetime64[ns]
 7   order_estimated_delivery_date  99441 non-null  datetime64[ns]
dtypes: datetime64[ns](5), object(3)
memory usage: 6.1+ MB


### Analyzing orders_reviews table

In [97]:
order_reviews.head()        # General view

,review_id,order_id,review_score,review_comment_title,review_comment_message,review_creation_date,review_answer_timestamp
0,7bc2406110b926393aa56f80a40eba40,73fc7af87114b39712e6da79b0a377eb,4,None,None,2018-01-18 00:00:00,2018-01-18 21:46:59
1,80e641a11e56f04c1ad469d5645fdfde,a548910a1c6147796b98fdf73dbeba33,5,None,None,2018-03-10 00:00:00,2018-03-11 03:05:13
2,228ce5500dc1d8e020d8d1322874b6f0,f9e4b658b201a9f2ecdecbb34bed034b,5,None,None,2018-02-17 00:00:00,2018-02-18 14:36:24
3,e64fb393e7b32834bb789ff8bb30750e,658677c97b385a9be170737859d3511b,5,None,Recebi bem antes do prazo estipulado.,2017-04-21 00:00:00,2017-04-21 22:02:06
4,f7c4243c7fe1938f181bec41a392bdeb,8e6bfb81e283fa7e4f11123a3fb894f1,5,None,Parabéns lojas lannister adorei comprar pela I...,2018-03-01 00:00:00,2018-03-02 10:26:53


In [98]:
order_reviews.info()    # General observation

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99224 entries, 0 to 99223
Data columns (total 7 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   review_id                99224 non-null  object
 1   order_id                 99224 non-null  object
 2   review_score             99224 non-null  object
 3   review_comment_title     11566 non-null  object
 4   review_comment_message   40968 non-null  object
 5   review_creation_date     99224 non-null  object
 6   review_answer_timestamp  99224 non-null  object
dtypes: object(7)
memory usage: 5.3+ MB


In [99]:
# Primary analysis
for column in order_reviews:
    print(f"{column} : {len(order_reviews[column].unique())}")
print("Shape: ", order_reviews.shape)

review_id : 98410
order_id : 98673
review_score : 5
review_comment_title : 4511
review_comment_message : 36156
review_creation_date : 636
review_answer_timestamp : 98248
Shape:  (99224, 7)


In [100]:
# Duplicate values exist in review_id and order_id columns

In [101]:
# Creating group keys of order_id and review_id
dup_counts = order_reviews.groupby(
    ["order_id", "review_id"]
).size().reset_index(name="count")

dup_counts[dup_counts["count"] > 1]

,order_id,review_id,count


In [102]:
# So there are no order_id, review_id pairs with multiple count

In [103]:
# Checking duplicate review_ids
dup_counts = order_reviews.groupby ("review_id").size().reset_index(name="counts")
dup_counts[dup_counts["counts"]>1]

,review_id,counts
31,00130cbe1f9d422698c812ed8ded1919,2
425,0115633a9c298b6a98bcbe4eee75345f,2
568,0174caf0ee5964646040cd94e15ac95e,2
574,017808d29fd1f942d97e50184dfb4c13,2
884,0254bd905dc677a6078990aad3331a36,2
...,...,...
97594,fde2e6abaf5bb64f7407a44741c24dec,2
97601,fde5986d35c89aa1b6ce4149de82a0d3,2
97794,fe5c833752953fed3209646f1f63b53c,2
98105,ff2fc9e68f8aabfbe18d710b83aabd30,2


In [104]:
order_reviews[order_reviews["review_id"]=="00130cbe1f9d422698c812ed8ded1919"]

,review_id,order_id,review_score,review_comment_title,review_comment_message,review_creation_date,review_answer_timestamp
29841,00130cbe1f9d422698c812ed8ded1919,04a28263e085d399c97ae49e0b477efa,1,None,"O cartucho ""original HP"" 60XL não é reconhecid...",2018-03-07 00:00:00,2018-03-20 18:08:23
46678,00130cbe1f9d422698c812ed8ded1919,dfcdfc43867d1c1381bfaf62d6b9c195,1,None,"O cartucho ""original HP"" 60XL não é reconhecid...",2018-03-07 00:00:00,2018-03-20 18:08:23


In [105]:
order_reviews[order_reviews["review_id"]=="0115633a9c298b6a98bcbe4eee75345f"]

,review_id,order_id,review_score,review_comment_title,review_comment_message,review_creation_date,review_answer_timestamp
63193,0115633a9c298b6a98bcbe4eee75345f,0c9850b2c179c1ef60d2855e2751d1fa,5,None,None,2017-09-21 00:00:00,2017-09-26 03:27:47
90677,0115633a9c298b6a98bcbe4eee75345f,78a4201f58af3463bdab842eea4bc801,5,None,None,2017-09-21 00:00:00,2017-09-26 03:27:47


In [106]:
# The same review_id has been assigned to multiple order_id. This is a data integrity issue

In [107]:
# Checking duplicate order_ids
dup_counts_orders = order_reviews.groupby("order_id").size().reset_index(name="count")
dup_counts_orders[dup_counts_orders["count"]>1]

,order_id,count
84,0035246a40f520710769010f752e7507,2
461,013056cfe49763c6f66bda03396c5ee3,2
556,0176a6846bcb3b0d3aa3116a9a768597,2
835,02355020fd0a40a0d56df9f6ff060413,2
985,029863af4b968de1e5d6a82782e662f5,2
...,...,...
97779,fd95ae805c63c534f1a64589e102225e,2
97943,fe041ba1c9f54016432fa6ee91709dbc,2
98480,ff763b73e473d03c321bcd5a053316e8,2
98497,ff850ba359507b996e8b2fbb26df8d03,2


In [108]:
order_reviews[order_reviews["order_id"] == "0035246a40f520710769010f752e7507"]

,review_id,order_id,review_score,review_comment_title,review_comment_message,review_creation_date,review_answer_timestamp
22423,2a74b0559eb58fc1ff842ecc999594cb,0035246a40f520710769010f752e7507,5,None,Estou acostumada a comprar produtos pelo barat...,2017-08-25 00:00:00,2017-08-29 21:45:57
25612,89a02c45c340aeeb1354a24e7d4b2c1e,0035246a40f520710769010f752e7507,5,None,None,2017-08-29 00:00:00,2017-08-30 01:59:12


In [109]:
# Dedupe order_id, keep the latest review

order_reviews = order_reviews.sort_values("review_answer_timestamp")
order_reviews = order_reviews.drop_duplicates(subset="order_id", keep="last")

In [110]:
# sanity check
order_reviews.shape

(98673, 7)

In [111]:
# Create a boolean flag column
order_reviews["has_reviews"] = order_reviews["review_comment_message"].notna()

In [112]:
# Normalizing review_id column
order_reviews["review_id"] = (
    order_reviews["review_id"]
    .str.lower()
    .str.strip()
    .str.replace(r"s\+", "", regex=True)
)

# Normalizing order_id column
order_reviews["order_id"] = (
    order_reviews["order_id"]
    .str.lower()
    .str.strip()
    .str.replace(r"s\+", "", regex=True)
)

# Normalizing review_comment_title column
order_reviews["review_comment_title"] = (
    order_reviews["review_comment_title"]
    .str.lower()
    .str.strip()
    .str.replace(r"s\+", " ", regex=True)
)

# Normalizing review_comment_message column
order_reviews["review_comment_message"] = (
    order_reviews["review_comment_message"]
    .str.lower()
    .str.strip()
    .str.replace(r"s\+", " ", regex=True)
)

In [113]:
order_reviews.head()

,review_id,order_id,review_score,review_comment_title,review_comment_message,review_creation_date,review_answer_timestamp,has_reviews
37547,6916ca4502d6d3bfd39818759d55d536,bfbd0f9bdef84302105ad712db648a6c,1,None,nao recebi o produto e nem resposta da empresa,2016-10-06 00:00:00,2016-10-07 18:32:28,True
5503,49f695dffa457eaba90d388a5c37e942,e5215415bb6f76fe3b7cb68103a0d1c0,1,None,"produto não chegou,e já passou o prazo de entrega",2016-10-09 00:00:00,2016-10-11 14:31:29,True
60439,743d98b1a4782f0646898fc915ef002a,e2144124f98f3bf46939bc5183104041,4,None,None,2016-10-15 00:00:00,2016-10-16 03:20:17,False
28076,53752edb26544dd41c1209f582c9c589,b8b9d7046c083150cb5360b83a8ebb51,5,None,o pedido foi entregue antes do prazo pr0metido,2016-10-16 01:00:00,2016-10-16 15:45:11,True
41042,b2d5d8db2a841d27a72e4c06c6212368,9aa3197e4887919fde0307fc23601d7a,4,None,só chegou uma parte do pedido ate agora..,2016-10-15 00:00:00,2016-10-17 21:02:49,True


In [114]:
order_reviews.info()

<class 'pandas.core.frame.DataFrame'>
Index: 98673 entries, 37547 to 80582
Data columns (total 8 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   review_id                98673 non-null  object
 1   order_id                 98673 non-null  object
 2   review_score             98673 non-null  object
 3   review_comment_title     11548 non-null  object
 4   review_comment_message   40766 non-null  object
 5   review_creation_date     98673 non-null  object
 6   review_answer_timestamp  98673 non-null  object
 7   has_reviews              98673 non-null  bool  
dtypes: bool(1), object(7)
memory usage: 6.1+ MB


In [115]:
# Changing the data types
order_reviews["review_score"] = order_reviews["review_score"].astype(int)
order_reviews["review_creation_date"] = pd.to_datetime(order_reviews["review_creation_date"], errors= "coerce")
order_reviews["review_answer_timestamp"] = pd.to_datetime(order_reviews["review_answer_timestamp"], errors= "coerce")

In [116]:
order_reviews.info()    # General observation

<class 'pandas.core.frame.DataFrame'>
Index: 98673 entries, 37547 to 80582
Data columns (total 8 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   review_id                98673 non-null  object        
 1   order_id                 98673 non-null  object        
 2   review_score             98673 non-null  int32         
 3   review_comment_title     11548 non-null  object        
 4   review_comment_message   40766 non-null  object        
 5   review_creation_date     98673 non-null  datetime64[ns]
 6   review_answer_timestamp  98673 non-null  datetime64[ns]
 7   has_reviews              98673 non-null  bool          
dtypes: bool(1), datetime64[ns](2), int32(1), object(4)
memory usage: 5.7+ MB


### Analyzing order_items dataset

In [117]:
order_items.head() # General view

,order_id,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value
0,00010242fe8c5a6d1ba2dd792cb16214,1,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,2017-09-19 09:45:35,58.90,13.29
1,00018f77f2f0320c557190d7a144bdd3,1,e5f2d52b802189ee658865ca93d83a8f,dd7ddc04e1b6c2c614352b383efe2d36,2017-05-03 11:05:13,239.90,19.93
2,000229ec398224ef6ca0657da4fc703e,1,c777355d18b72b67abbeef9df44fd0fd,5b51032eddd242adc84c38acab88f23d,2018-01-18 14:48:30,199.00,17.87
3,00024acbcdf0a6daa1e931b038114c75,1,7634da152a4610f1595efa32f14722fc,9d7a1d34a5052409006425275ba1c2b4,2018-08-15 10:10:18,12.99,12.79
4,00042b26cf59d7ce69dfabb4e55b4fd9,1,ac6c3623068f30de03045865e4e10089,df560393f3a51e74553ab94004ba5c87,2017-02-13 13:57:51,199.90,18.14


In [118]:
# Primary analysis
for column in order_items:
    print(f"{column} : {len(order_items[column].unique())}")
print("Shape: ", order_items.shape)

order_id : 98666
order_item_id : 21
product_id : 32951
seller_id : 3095
shipping_limit_date : 93318
price : 5968
freight_value : 6999
Shape:  (112650, 7)


In [119]:
# General observation
order_items.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 112650 entries, 0 to 112649
Data columns (total 7 columns):
 #   Column               Non-Null Count   Dtype 
---  ------               --------------   ----- 
 0   order_id             112650 non-null  object
 1   order_item_id        112650 non-null  object
 2   product_id           112650 non-null  object
 3   seller_id            112650 non-null  object
 4   shipping_limit_date  112650 non-null  object
 5   price                112650 non-null  object
 6   freight_value        112650 non-null  object
dtypes: object(7)
memory usage: 6.0+ MB


In [120]:
# Change the data types
order_items["order_item_id"] = order_items["order_item_id"].astype(int)
order_items["shipping_limit_date"] = pd.to_datetime(order_items["shipping_limit_date"], errors="coerce")
order_items["price"] = pd.to_numeric(order_items["price"], errors="coerce")
order_items["freight_value"] = pd.to_numeric(order_items["freight_value"], errors="coerce")

In [121]:
# Check null values
order_items.isnull().sum()

order_id               0
order_item_id          0
product_id             0
seller_id              0
shipping_limit_date    0
price                  0
freight_value          0
dtype: int64

In [122]:
# Normalizing the order_id column
order_items["order_id"] = (
    order_items["order_id"]
    .str.lower()
    .str.strip()
    .str.replace(r"s\+", "", regex=True)
)

#Normalizing the product_id column
order_items["product_id"] = (
    order_items["product_id"]
    .str.lower()
    .str.strip()
    .str.replace(r"\s+", "", regex=True)
)

# Normalizing the seller_id column
order_items["seller_id"] = (
    order_items["seller_id"]
    .str.lower()
    .str.strip()
    .str.replace(r"\s+", "", regex=True)
)

### Analyzing order_payments table

In [123]:
order_payments.head() # General view

,order_id,payment_sequential,payment_type,payment_installments,payment_value
0,b81ef226f3fe1789b1e8b2acac839d17,1,credit_card,8,99.33
1,a9810da82917af2d9aefd1278f1dcfa0,1,credit_card,1,24.39
2,25e8ea4e93396b6fa0d3dd708e76c1bd,1,credit_card,1,65.71
3,ba78997921bbcdc1373bb41e913ab953,1,credit_card,8,107.78
4,42fdf880ba16b47b59251dd489d4441a,1,credit_card,2,128.45


In [124]:
# Primary analysis
for column in order_payments:
    print(f"{column} : {len(order_payments[column].unique())}")
print("Shape: ", order_payments.shape)

order_id : 99440
payment_sequential : 29
payment_type : 5
payment_installments : 24
payment_value : 29077
Shape:  (103886, 5)


In [125]:
# Change the data types
order_payments["payment_sequential"] = order_payments["payment_sequential"].astype(int)
order_payments["payment_installments"] = order_payments["payment_installments"].astype(int)
order_payments["payment_value"] = order_payments["payment_value"].astype(float)

In [126]:
# General observation
order_payments.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 103886 entries, 0 to 103885
Data columns (total 5 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   order_id              103886 non-null  object 
 1   payment_sequential    103886 non-null  int32  
 2   payment_type          103886 non-null  object 
 3   payment_installments  103886 non-null  int32  
 4   payment_value         103886 non-null  float64
dtypes: float64(1), int32(2), object(2)
memory usage: 3.2+ MB


In [127]:
order_payments.head()

,order_id,payment_sequential,payment_type,payment_installments,payment_value
0,b81ef226f3fe1789b1e8b2acac839d17,1,credit_card,8,99.33
1,a9810da82917af2d9aefd1278f1dcfa0,1,credit_card,1,24.39
2,25e8ea4e93396b6fa0d3dd708e76c1bd,1,credit_card,1,65.71
3,ba78997921bbcdc1373bb41e913ab953,1,credit_card,8,107.78
4,42fdf880ba16b47b59251dd489d4441a,1,credit_card,2,128.45


In [128]:
# Normalizing order_id column
order_payments["order_id"] = (
    order_payments["order_id"]
    .str.lower()
    .str.strip()
    .str.replace(r"s\+", "", regex=True)
)

# Normalizing payment_type column
order_payments["payment_type"] = (
    order_payments["payment_type"]
    .str.lower()
    .str.strip()
    .str.replace(r"s\+", "", regex=True)
)

In [129]:
# url encode and plug into sql alchemy
quoted = urllib.parse.quote_plus(conn_str)
engine = create_engine(f"mssql+pyodbc:///?odbc_connect={quoted}")

In [130]:
# Defining data types
dtype_map = {
    'customers': {
        'customer_id': VARCHAR(length=50),
        'customer_unique_id': VARCHAR(length=50),
        'customer_zip_code_prefix': VARCHAR(length=10),
        'customer_city': NVARCHAR(length=100),
        'customer_state': VARCHAR(length=2),
    },
    'orders': {
        'order_id': VARCHAR(length=50),
        'customer_id': VARCHAR(length=50),
        'order_status': VARCHAR(length=20),
        'order_purchase_timestamp': DateTime(),
        'order_approved_at': DateTime(),
        'order_delivered_carrier_date': DateTime(),
        'order_delivered_customer_date': DateTime(),
        'order_estimated_delivery_date': DateTime(),
    },
    'order_items': {
        'order_id': VARCHAR(length=50),
        'order_item_id': Integer(),
        'product_id': VARCHAR(length=50),
        'seller_id': VARCHAR(length=50),
        'shipping_limit_date': DateTime(),
        'price': Float(),
        'freight_value': Float(),
    },
    'order_payments': {
        'order_id': VARCHAR(length=50),
        'payment_sequential': Integer(),
        'payment_type': VARCHAR(length=20),
        'payment_installments': Integer(),
        'payment_value': Float(),
    },
    'order_reviews': {
        'order_id': VARCHAR(length=50),
        'review_score': Integer(),
        'review_comment_title': NVARCHAR(length='max'),
        'review_comment_message': NVARCHAR(length='max'),
        'review_creation_date': DateTime(),
        'review_answer_timestamp': DateTime(),
        'has_review': Boolean(),
    },
    'products': {
        'product_id': VARCHAR(length=50),
        'product_category_name': NVARCHAR(length=100),
        'product_name_length': Integer(),
        'product_description_length': Integer(),
        'product_photos_qty': Integer(),
        'product_weight_g': Float(),
        'product_length_cm': Float(),
        'product_height_cm': Float(),
        'product_width_cm': Float(),
        'product_category_name_english': NVARCHAR(length=100),
    },
    'sellers': {
        'seller_id': VARCHAR(length=50),
        'seller_zip_code_prefix': VARCHAR(length=10),
        'seller_city': NVARCHAR(length=100),
        'seller_state': VARCHAR(length=2),
    },
    'geolocation': {
        'geolocation_zip_code_prefix': VARCHAR(length=10),
        'geolocation_lat': Float(),
        'geolocation_lng': Float(),
        'geolocation_city': NVARCHAR(length=100),
        'geolocation_state': VARCHAR(length=2),
    },
    'category_translation': {
        'product_category_name': NVARCHAR(length=100),
        'product_category_name_english': NVARCHAR(length=100),
    },
    'geo_grp' : {
        'geolocation_zip_code_prefix' : VARCHAR(length=10),
        'geolocation_city' : NVARCHAR(length=100),
        'geolocation_state' : VARCHAR(2),
    }
}

In [131]:
# Dictionary of cleaned data frames
clean_dfs = {
    "customers" : customers,
    "sellers" : sellers,
    "products" : products,
    "orders" : orders,
    "order_items" : order_items,
    "order_reviews" : order_reviews,
    "order_payments" : order_payments,
    "geolocation" : geolocation,
    "cat_translation" : cat_translation,
    "geo_grp" : geo_grp
}

In [133]:
for name, table in clean_dfs.items():
    table.to_sql(
        name = name,
        schema = "analytics",
        con = engine,
        if_exists = "replace",
        index = False,
        dtype = dtype_map.get(name)
    )

    print(f"{name} : {len(table):,} rows exported to analytics.{name}")
    

customers : 99,441 rows exported to analytics.customers
sellers : 3,095 rows exported to analytics.sellers
products : 32,951 rows exported to analytics.products
orders : 99,441 rows exported to analytics.orders
order_items : 112,650 rows exported to analytics.order_items
order_reviews : 98,673 rows exported to analytics.order_reviews
order_payments : 103,886 rows exported to analytics.order_payments
geolocation : 1,000,163 rows exported to analytics.geolocation
cat_translation : 71 rows exported to analytics.cat_translation
geo_grp : 19,616 rows exported to analytics.geo_grp
